<a href="https://colab.research.google.com/github/pvictor728/Classificacao-choro-de-bebe/blob/main/audioClassification_Def.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Importar bibliotecas necessárias

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
%pip install tensorflow_io

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 28.0/28.0 MB 47.8 MB/s eta 0:00:00


In [ ]:
import os
import pathlib

import matplotlib.pyplot as plt
import tensorflow as tf
import tensorflow_io as tfio
import IPython.display as ipd
import datetime
from pathlib import Path
from imblearn.under_sampling import RandomUnderSampler
from imblearn.over_sampling import RandomOverSampler
import random
import numpy as np
from imblearn.over_sampling import SMOTE
import librosa
from sklearn.model_selection import cross_val_score
from sklearn.tree import DecisionTreeClassifier
from sklearn.metrics import make_scorer, accuracy_score, classification_report
from sklearn.preprocessing import normalize
from sklearn.model_selection import train_test_split

default_dir = '/content/drive/MyDrive/UFRN/2023.1/A.M./Dataset'

Passo dois: leitura e classficação dos áudios

In [ ]:
def load_audio(file_path):
    audio_binary = tf.io.read_file(file_path)
    audio, sample_rate = tf.audio.decode_wav(audio_binary)
    waveform = tf.squeeze(audio, axis=-1)
    normalized_waveform = waveform / tf.reduce_max(tf.abs(waveform))
    return normalized_waveform, sample_rate

def load_audio_files(path: str, label:str):

    dataset = []
    walker = sorted(str(p) for p in Path(path).glob(f'*.wav'))

    for i, file_path in enumerate(walker):
        waveform, sample_rate = load_audio(file_path)
        dataset.append((waveform, label))

    return dataset

In [ ]:
os.chdir(default_dir)
labels = [name for name in os.listdir('.') if os.path.isdir(name)]
print(f'Total Labels: {len(labels)}')
print(f'Label Names: {labels}')

Total Labels: 5
Label Names: ['tired', 'belly_pain', 'hungry', 'discomfort', 'data']


In [ ]:
filenames = tf.io.gfile.glob(str(default_dir) + '/*/*')
num_samples = len(filenames)
print('Number of total examples:', num_samples)

Number of total examples: 451


In [ ]:
trainset_belly_pain = load_audio_files('./belly_pain', 'belly_pain')
trainset_discomfort = load_audio_files('./discomfort', 'discomfort')
trainset_hungry = load_audio_files('./hungry', 'hungry')
trainset_tired = load_audio_files('./tired', 'tired')

In [ ]:
# Concatenar todos os conjuntos em um único conjunto de dados
all_data = (
        trainset_belly_pain +
        trainset_discomfort +
        trainset_hungry +
        trainset_tired
)

# Separar os recursos (waveform) e os rótulos (etiquetas)
X = np.array([data[0] for data in all_data])  # Recursos (waveform)
y = np.array([data[1] for data in all_data])  # Rótulos (etiquetas)
X = X.reshape(-1, 1)

# Realizar o balanceamento usando RandomOverSampler
ros = RandomOverSampler()
X_balanced, y_balanced = ros.fit_resample(X, y)

trainset_belly_pain = X_balanced[y_balanced == 'belly_pain']
trainset_discomfort = X_balanced[y_balanced == 'discomfort']
trainset_hungry = X_balanced[y_balanced == 'hungry']
trainset_tired = X_balanced[y_balanced == 'tired']

<ipython-input-135-3420cdfd3dee>:10: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray.
  X = np.array([data[0] for data in all_data])  # Recursos (waveform)


In [ ]:
def get_spectrogram(waveform):
    frame_length = 255
    frame_step = 128

    audio_length = tf.shape(waveform)[0]
    padding_length = tf.maximum(8000 - audio_length, 0)
    zero_padding = tf.zeros([padding_length], dtype=tf.float32)

    equal_length_waveform = tf.concat([waveform, zero_padding], 0)

    spect = tfio.audio.spectrogram(input=equal_length_waveform, nfft=frame_length, window=frame_length, stride=frame_step)

    spectrogram = tf.signal.stft(equal_length_waveform, frame_length=frame_length, frame_step=frame_step)
    spectrogram = tf.abs(spectrogram)

    return spectrogram, spect

In [ ]:
def create_images(dataset, label_dir):
    # make directory
    test_directory = f'./data/test/{label_dir}/'
    train_directory = f'./data/train/{label_dir}/'

    os.makedirs(test_directory, mode=0o777, exist_ok=True)
    os.makedirs(train_directory, mode=0o777, exist_ok=True)

    dataset_length = len(dataset)
    split_index = int(dataset_length * 0.2)

    for i, data in enumerate(dataset):

        waveform = data[0]
        spectrogram, spect = get_spectrogram(waveform)

        # Split test and train images by 20%
        if i < split_index:
            plt.imsave(f'./data/test/{label_dir}/spec_img{i}.png', spectrogram.numpy(), cmap='gray')
        else:
            plt.imsave(f'./data/train/{label_dir}/spec_img{i}.png', spectrogram.numpy(), cmap='gray')



In [ ]:
create_images(trainset_belly_pain, 'belly_pain')
create_images(trainset_discomfort, 'discomfort')
create_images(trainset_hungry, 'hungry')
create_images(trainset_tired, 'tired')

In [ ]:
train_directory = './data/train/'
test_directory = './data/test/'

train_ds = tf.keras.preprocessing.image_dataset_from_directory(
    train_directory, labels='inferred', label_mode='int', image_size=(425, 129), seed=123,
    validation_split=0.2, subset='validation')

test_ds = tf.keras.preprocessing.image_dataset_from_directory(
    test_directory, labels='inferred', label_mode='int', image_size=(425, 129),
    validation_split=None, subset=None)

Found 1224 files belonging to 4 classes.
Using 244 files for validation.
Found 304 files belonging to 4 classes.


In [ ]:
# Configuração do modelo
num_classes = 4
img_height = 425
img_width = 129

model = tf.keras.Sequential([
  tf.keras.layers.experimental.preprocessing.Rescaling(1./255, input_shape=(img_height, img_width, 3)),
  tf.keras.layers.Conv2D(16, 3, padding='same', activation='relu'),
  tf.keras.layers.MaxPooling2D(),
  tf.keras.layers.Conv2D(32, 3, padding='same', activation='relu'),
  tf.keras.layers.MaxPooling2D(),
  tf.keras.layers.Conv2D(64, 3, padding='same', activation='relu'),
  tf.keras.layers.MaxPooling2D(),
  tf.keras.layers.Flatten(),
  tf.keras.layers.Dense(128, activation='relu'),
  tf.keras.layers.Dense(num_classes)
])

In [ ]:
learning_rate = 0.125

loss_fn = tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True)
optimizer = tf.keras.optimizers.SGD(learning_rate)
metrics = ['accuracy']
model.compile(optimizer, loss_fn, metrics)

In [ ]:
# Set the epocks
epochs = 5
print('\nFitting:')

# Train the model.
history = model.fit(train_ds, epochs=epochs)


Fitting:
Epoch 1/5
8/8 [==============================] - 19s 2s/step - loss: 1.3683 - accuracy: 0.3238
Epoch 2/5
8/8 [==============================] - 17s 2s/step - loss: 1.2750 - accuracy: 0.4262
Epoch 3/5
8/8 [==============================] - 16s 2s/step - loss: 1.1824 - accuracy: 0.6352
Epoch 4/5
8/8 [==============================] - 16s 2s/step - loss: 0.7849 - accuracy: 0.7172
Epoch 5/5
8/8 [==============================] - 16s 2s/step - loss: 0.3047 - accuracy: 0.9180


Testando

In [ ]:
correct = 0
batch_size = 0
for batch_num, (X, Y) in enumerate(test_ds):
    batch_size = len(Y)
    pred = model.predict(X)
    for i in range(batch_size):
        predicted = np.argmax(pred[i], axis=-1)
        actual = Y[i]
        #print(f'predicted {predicted}, actual {actual}')
        if predicted == actual:
            correct += 1
    break

print(f'Number correct: {correct} out of {batch_size}')
print(f'Accuracy {correct / batch_size}')

1/1 [==============================] - 1s 1s/step
Number correct: 29 out of 32
Accuracy 0.90625


# **Modelo árvore de decisão**

In [ ]:
def extract_features(waveform, sample_rate):
    # Extrair os recursos do áudio usando a biblioteca librosa
    # Exemplo: MFCCs (Mel-frequency cepstral coefficients)
    mfccs = librosa.feature.mfcc(y=waveform.numpy(), sr=sample_rate.numpy(), n_mfcc=13)
    return np.mean(mfccs.T, axis=0)  # Retornar a média dos MFCCs como representação do áudio

In [ ]:
def load_audio(file_path):
    audio_binary = tf.io.read_file(file_path)
    audio, sample_rate = tf.audio.decode_wav(audio_binary)
    waveform = tf.squeeze(audio, axis=-1)
    return waveform, sample_rate

def load_audio_files(path: str, label: str):
    dataset = []
    walker = sorted(str(p) for p in Path(path).glob(f'*.wav'))

    for i, file_path in enumerate(walker):
        waveform, sample_rate = load_audio(file_path)
        features = extract_features(waveform, sample_rate)
        dataset.append((features, label))

    return dataset

In [ ]:
# Dividir os dados em conjuntos de treinamento e teste (80% treinamento, 20% teste)
X_train, X_test, y_train, y_test = train_test_split(X_balanced, y_balanced_encoded, test_size=0.2, random_state=42)

In [ ]:
# Criar uma instância do classificador de árvore de decisão
decision_tree_classifier = DecisionTreeClassifier()
# Treinar o modelo com os dados de treinamento
decision_tree_classifier.fit(X_train, y_train)

In [ ]:
# Realizar a predição no conjunto de teste
y_pred = decision_tree_classifier.predict(X_test)

# Calcular a acurácia do modelo
accuracy = accuracy_score(y_test, y_pred)
print(f'Acurácia do modelo: {accuracy}')

# Exibir o relatório de classificação
class_names = label_encoder.classes_
report = classification_report(y_test, y_pred, target_names=class_names)
print('Relatório de Classificação:')
print(report)

In [ ]:
# Salvar o modelo treinado em um arquivo
model_filename = 'decision_tree_model.joblib'
joblib.dump(decision_tree_classifier, model_filename)